Step 1: Import file handling libraries

In [ ]:
from google.colab import files
import os
import shutil

Step 2: Import audio processing libraries

In [ ]:
from pydub import AudioSegment
import librosa
import soundfile as sf

Step 3: Import concurrency and progress tracking tools

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

Step 4: Import numerical and plotting libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Step 5: Import machine learning libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Step 6: Upload files from your local system

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aryamang2004","key":"6ac506b243e65835dd221996b83cce54"}'}

Step 7: Set up Kaggle API credentials

In [ ]:
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 600)

Step 8: Download datasets from Kaggle

In [ ]:
!kaggle datasets download -d aryamang2004/labeled-dementia
!kaggle datasets download -d aryamang2004/labeled-control
!kaggle datasets download -d aryamang2004/unlabeled

Dataset URL: https://www.kaggle.com/datasets/aryamang2004/labeled-dementia
License(s): unknown
Dataset URL: https://www.kaggle.com/datasets/aryamang2004/labeled-control
License(s): unknown
Dataset URL: https://www.kaggle.com/datasets/aryamang2004/unlabeled
License(s): unknown


Step 9: Create directories to store datasets

In [ ]:
!mkdir -p data/labeled-dementia
!mkdir -p data/labeled-control
!mkdir -p data/unlabeled

Step 10: Unzip the downloaded dataset files

In [ ]:
!unzip labeled-dementia.zip -d data/labeled-dementia
!unzip labeled-control.zip -d data/labeled-control
!unzip unlabeled.zip -d data/unlabeled

Archive:  labeled-dementia.zip
  inflating: data/labeled-dementia/Labeled (Dementia)/001-0.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/001-2.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/003-0.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/005-0.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/005-2.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/007-1.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/007-3.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/010-0.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/010-1.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/010-2.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/010-3.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/010-4.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/014-2.mp3  
  inflating: data/labeled-dementia/Labeled (Dementia)/016-0.mp3  
  inflating: data/labeled-dementia/Labeled (D

Step 11: Define a function to count audio files

In [ ]:
def count_audio_files(path, exts=(".mp3")):

    return len([f for f in os.listdir(path) if f.endswith(exts)])

Step 12: Set dataset folder paths

In [ ]:
control_path = "/content/data/labeled-control/Labeled (Control)"
dementia_path = "/content/data/labeled-dementia/Labeled (Dementia)"
unlabeled_path = "/content/data/unlabeled/Unlabeled"

Step 13: Count audio files in each dataset

In [ ]:
num_control = count_audio_files(control_path)
num_dementia = count_audio_files(dementia_path)
num_unlabeled = count_audio_files(unlabeled_path)

Step 14: Print the number of audio clips in each category

In [ ]:
print(f"Dementia Clips: {num_dementia}")
print(f"Control Clips: {num_control}")
print(f"Unlabeled Clips: {num_unlabeled}")

Dementia Clips: 100
Control Clips: 100
Unlabeled Clips: 500


Step 15: Install required audio dependencies

In [ ]:
!pip install pydub
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


 Step 16: Define function to convert a single MP3 to WAV

In [ ]:
def convert_single(file_info):
    src_file, src_dir, dest_dir = file_info

    try:
        src_path = os.path.join(src_dir, src_file)
        dest_path = os.path.join(dest_dir, src_file.replace('.mp3', '.wav'))

        if not os.path.exists(dest_path):
            audio = AudioSegment.from_mp3(src_path)
            audio.export(dest_path, format='wav')

    except Exception as e:
        print(f"Error with {src_file}: {e}")

Step 17: Define function to convert all MP3s using multithreading

In [ ]:
def convert_all_mp3s(src_dir, dest_dir, max_threads=8):
    os.makedirs(dest_dir, exist_ok=True)
    files = [f for f in os.listdir(src_dir) if f.endswith(".mp3")]
    file_infos = [(f, src_dir, dest_dir) for f in files]

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        list(tqdm(executor.map(convert_single, file_infos), total=len(file_infos)))

Step 18: Set source paths for original MP3 files

In [ ]:
dementia_src = "/content/data/labeled-dementia/Labeled (Dementia)"
control_src = "/content/data/labeled-control/Labeled (Control)"
unlabeled_src = "/content/data/unlabeled/Unlabeled"

Step 19: Convert MP3s to WAVs and save them

In [ ]:
convert_all_mp3s(dementia_src, "/content/processed-wav/dementia", max_threads=16)
convert_all_mp3s(control_src, "/content/processed-wav/control", max_threads=16)
convert_all_mp3s(unlabeled_src, "/content/processed-wav/unlabeled", max_threads=16)

100%|██████████| 500/500 [01:37<00:00,  5.12it/s]


Step 20: Define function to extract Mel spectrogram from audio

In [ ]:
def extract_mel_spec(file_path, sr=16000, n_mels=128, duration=5):
    y, _ = librosa.load(file_path, sr=sr, duration=duration)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    return mel_spec_db

Step 21: Define function to save Mel spectrograms from WAV files

In [ ]:
def extract_all_mels(src_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)

    for file in tqdm(os.listdir(src_dir)):

        if not file.endswith('.wav'):

          continue
        full_path = os.path.join(src_dir, file)
        mel = extract_mel_spec(full_path)
        out_path = os.path.join(dest_dir, file.replace('.wav', '.npy'))
        np.save(out_path, mel)

Step 22: Run Mel spectrogram extraction for all datasets

In [ ]:
extract_all_mels("/content/processed-wav/dementia", "/content/processed-specs/dementia")
extract_all_mels("/content/processed-wav/control", "/content/processed-specs/control")
extract_all_mels("/content/processed-wav/unlabeled", "/content/processed-specs/unlabeled")

100%|██████████| 500/500 [00:08<00:00, 56.22it/s]


Step 23: Split training data into training and validation sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Step 24: Define the CNN model architecture

In [ ]:
def build_cnn(input_shape=(128, 157, 1)):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

Step 25: Compile and return the CNN model

In [ ]:
 model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

Step 26: Set early stopping and learning rate scheduler

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=0.00001)

Step 27: Build the CNN model

In [ ]:
model = build_cnn(input_shape=(128, 157, 1))

Step 28: Define function to load and preprocess spectrogram data

In [ ]:
def load_spectrogram_data(dementia_dir, control_dir):
    X = []
    y = []

    for file in os.listdir(dementia_dir):

        if file.endswith('.npy'):
            spec = np.load(os.path.join(dementia_dir, file))

            if spec.shape[1] < 157:
                pad_width = 157 - spec.shape[1]
                spec = np.pad(spec, ((0, 0), (0, pad_width)), mode='constant')

            X.append(spec)
            y.append(1)

    for file in os.listdir(control_dir):

        if file.endswith('.npy'):
            spec = np.load(os.path.join(control_dir, file))

            if spec.shape[1] < 157:
                pad_width = 157 - spec.shape[1]
                spec = np.pad(spec, ((0, 0), (0, pad_width)), mode='constant')

            X.append(spec)
            y.append(0)

    X = np.array(X)
    X = np.expand_dims(X, -1)
    y = np.array(y)

    return train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Step 29: Set paths for processed spectrogram data

In [ ]:
dementia_path = "/content/processed-specs/dementia"
control_path = "/content/processed-specs/control"

Step 30: Load training and testing data from spectrograms

In [ ]:
X_train, X_test, y_train, y_test = load_spectrogram_data(dementia_path, control_path)

Step 31: Initialize the CNN model

In [ ]:
model = build_cnn()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Step 32: Train the CNN model

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.4628 - loss: 1.2781 - val_accuracy: 0.5385 - val_loss: 1.1476 - learning_rate: 1.0000e-04
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.7864 - loss: 0.4861 - val_accuracy: 0.4615 - val_loss: 5.7735 - learning_rate: 1.0000e-04
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.9455 - loss: 0.1203 - val_accuracy: 0.4615 - val_loss: 3.4455 - learning_rate: 1.0000e-04
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.9502 - loss: 0.1124 - val_accuracy: 0.4615 - val_loss: 1.4988 - learning_rate: 5.0000e-05
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.9949 - loss: 0.0422 - val_accuracy: 0.5769 - val_loss: 0.7437 - learning_rate: 5.0000e-05
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.9932 - loss: 0.0241 - val_accuracy: 0.6538 - val_loss: 0.5940 - learning_rate: 5.0000e-05
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 20s 4s/step - accuracy: 0.9949 - loss: 0.0212 - val_

Step 33: Evaluate model on validation set

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 704ms/step - accuracy: 0.8462 - loss: 0.4323


Step 34: Print validation loss and accuracy

In [ ]:
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Loss: 0.4323
Validation Accuracy: 0.8462


Step 35: Print final training accuracy

In [ ]:
final_train_acc = history.history['accuracy'][-1]
print(f"Final Training Accuracy: {final_train_acc:.4f}")

Final Training Accuracy: 0.9688


Step 36: Print average training accuracy

In [ ]:
avg_train_acc = sum(history.history['accuracy']) / len(history.history['accuracy'])
print(f"Average Training Accuracy: {avg_train_acc:.4f}")

Average Training Accuracy: 0.9514


Step 37: Set path for unlabeled spectrogram data

In [ ]:
unlabeled_data_dir = '/content/processed-specs/unlabeled'

Step 38: Load unlabeled spectrograms and make predictions

In [ ]:
if os.path.exists(unlabeled_data_dir):
    files = [f for f in os.listdir(unlabeled_data_dir) if f.endswith('.npy')]
    print(f"Found {len(files)} .npy files in the directory.")

    X_unlabeled = []

    for file in files:
        file_path = os.path.join(unlabeled_data_dir, file)
        data = np.load(file_path)

        if data.shape == (128, 157):
            X_unlabeled.append(data[..., np.newaxis])

    X_unlabeled = np.array(X_unlabeled)
    print(f"Shape of X_unlabeled: {X_unlabeled.shape}")

    if X_unlabeled.shape[0] > 0:
        predictions = model.predict(X_unlabeled)
        print(f"Predictions: {predictions}")

    else:
        print("No valid data found to make predictions.")

else:
    print("Directory not found. Please check the path.")

Found 500 .npy files in the directory.
Shape of X_unlabeled: (302, 128, 157, 1)
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 991ms/step
Predictions: [[0.9563874 ]
 [0.86597574]
 [0.9367032 ]
 [0.9741651 ]
 [0.6763028 ]
 [0.50558835]
 [0.71402556]
 [0.8987394 ]
 [0.9527493 ]
 [0.95920646]
 [0.5301215 ]
 [0.94786584]
 [0.685562  ]
 [0.859101  ]
 [0.5552794 ]
 [0.49050942]
 [0.76988935]
 [0.71375525]
 [0.94192034]
 [0.925238  ]
 [0.7464245 ]
 [0.699568  ]
 [0.97717935]
 [0.9151375 ]
 [0.9735918 ]
 [0.95680857]
 [0.8213945 ]
 [0.44634128]
 [0.9325564 ]
 [0.88120174]
 [0.7618474 ]
 [0.8424057 ]
 [0.7323867 ]
 [0.56398934]
 [0.8872865 ]
 [0.75637704]
 [0.21315183]
 [0.44900525]
 [0.8918565 ]
 [0.86853385]
 [0.8283242 ]
 [0.7884174 ]
 [0.9691951 ]
 [0.7346874 ]
 [0.25822714]
 [0.6574965 ]
 [0.9462642 ]
 [0.6843257 ]
 [0.91804993]
 [0.98788965]
 [0.77073634]
 [0.8002476 ]
 [0.66727704]
 [0.7173035 ]
 [0.67610526]
 [0.9071235 ]
 [0.72883075]
 [0.79390746]
 [0.5283479 ]
 [0.28744063]
 [0.39405653]
 [0.4112461

Step 39: Convert model predictions to binary labels

In [ ]:
binary_preds = (predictions > 0.5).astype(int)

Step 40: Count predicted dementia and control samples

In [ ]:
dementia_count = np.sum(binary_preds)
control_count = len(binary_preds) - dementia_count

Step 41: Print prediction summary for unlabeled data

In [ ]:
print(f"Dementia: {dementia_count}, Control: {control_count}")

Dementia: 244, Control: 58


Step 42: Calculate pseudo-labeling confidence accuracy

In [ ]:
confident_preds = (predictions > 0.6) | (predictions < 0.4)
pseudo_accuracy = np.sum(confident_preds) / len(predictions)

Step 43: Print pseudo accuracy percentage

In [ ]:
print(f"Pseudo Accuracy (confident predictions): {pseudo_accuracy * 100:.2f}%")

Pseudo Accuracy (confident predictions): 87.75%
